In [8]:
import os
import os.path
import numpy as np
import h5py
import time
import matplotlib.pyplot as plt

import skimage.io as sio

In [9]:
# Define which folders to parse:
root_dir='/esat/opal/kkelchte/docker_home/pilot_data/canyon_forest_sandbox'
# root_dir='/esat/opal/kkelchte/docker_home/pilot_data/small'
runs=sorted([root_dir+'/'+d for d in os.listdir(root_dir) if d.startswith('0')])
print len(runs),' runs.'
print [os.path.basename(r) for r in runs]

300  runs.
['00000_canyon', '00000_forest', '00000_sandbox', '00001_canyon', '00001_forest', '00001_sandbox', '00002_canyon', '00002_forest', '00002_sandbox', '00003_canyon', '00003_forest', '00003_sandbox', '00004_canyon', '00004_forest', '00004_sandbox', '00005_canyon', '00005_forest', '00005_sandbox', '00006_canyon', '00006_forest', '00006_sandbox', '00007_canyon', '00007_forest', '00007_sandbox', '00008_canyon', '00008_forest', '00008_sandbox', '00009_canyon', '00009_forest', '00009_sandbox', '00010_canyon', '00010_forest', '00010_sandbox', '00011_canyon', '00011_forest', '00011_sandbox', '00012_canyon', '00012_forest', '00012_sandbox', '00013_canyon', '00013_forest', '00013_sandbox', '00014_canyon', '00014_forest', '00014_sandbox', '00015_canyon', '00015_forest', '00015_sandbox', '00016_canyon', '00016_forest', '00016_sandbox', '00017_canyon', '00017_forest', '00017_sandbox', '00018_canyon', '00018_forest', '00018_sandbox', '00019_canyon', '00019_forest', '00019_sandbox', '00020_c

In [10]:
root_name= 'data.hdf5'
# remove if it already exists
if os.path.isfile(root_dir+'/'+root_name):
    os.remove(root_dir+'/'+root_name)
f = h5py.File(root_dir+'/'+root_name, 'w')

In [11]:
# create a group for this run
for r in runs: f.create_group(os.path.basename(r))

In [13]:
def add_dir(group, data_type='RGB'):
    if os.path.isdir(r+'/'+data_type):
        # get an idea of the shape
        image_files=sorted([r+'/'+data_type+'/'+i for i in os.listdir(r+'/'+data_type)])
        img=sio.imread(image_files[0])
        try:
            img_array=np.zeros((len(image_files), img.shape[0], img.shape[1], img.shape[2]),dtype=np.uint8)
        except:
            img_array=np.zeros((len(image_files), img.shape[0], img.shape[1]),dtype=np.uint8)
        for i, imgf in enumerate(image_files): img_array[i]=sio.imread(imgf)
#         plt.imshow(img_array[-1])
#         plt.show()
        print('extracted {0}, shape: {1}'.format(data_type,img_array.shape))
        return group.create_dataset(data_type,data=img_array, compression="gzip", compression_opts=2)
#         return group.create_dataset(data_type,data=img_array, compression="lzf")
    else:
        print('could not find {0} in {1}.'.format(data_type,group.name))
        return []


In [14]:
def add_file(group, file_name='control_info'):
    if os.path.isfile(r+'/'+file_name+'.txt'):
        info_file=open(r+'/'+file_name+'.txt','r')
        info=np.asarray([[float(i) for i in l.split(' ')]  for l in info_file.readlines()])
        print('extracted {0}, shape: {1}'.format(file_name, info.shape))
#         return group.create_dataset(file_name,data=info, compression="gzip", compression_opts=9)
        return group.create_dataset(file_name,data=info, compression="lzf")
    else:
        print('could not find {0} in {1}.'.format(file_name,group.name))
        return []

In [ ]:
while [ len(runs) != 0 ]:
    r=runs.pop(0)
    print("[{1[2]}/{1[1]}_{1[3]:02d}:{1[4]:02d}]: {0} left".format(len(runs),tuple(time.localtime()[0:5])))
    add_dir(f[os.path.basename(r)],'RGB')
    add_dir(f[os.path.basename(r)],'Depth')
    add_dir(f[os.path.basename(r)],'Depth_predicted')
    add_file(f[os.path.basename(r)],'control_info')
    add_file(f[os.path.basename(r)],'collision_info')
    add_file(f[os.path.basename(r)],'collision_info_3')
    add_file(f[os.path.basename(r)],'collision_info_5')
    add_file(f[os.path.basename(r)],'collision_info_7')
    f.flush()

[6/4_10:28]: 299 left
extracted RGB, shape: (557, 360, 640, 3)
extracted Depth, shape: (556, 360, 640)
could not find Depth_predicted in /00000_canyon.
extracted control_info, shape: (1113, 7)
extracted collision_info, shape: (1113, 2)
could not find collision_info_3 in /00000_canyon.
could not find collision_info_5 in /00000_canyon.
could not find collision_info_7 in /00000_canyon.
[6/4_10:28]: 298 left
extracted RGB, shape: (720, 360, 640, 3)
extracted Depth, shape: (721, 360, 640)
could not find Depth_predicted in /00000_forest.
extracted control_info, shape: (1440, 7)
extracted collision_info, shape: (1440, 2)
could not find collision_info_3 in /00000_forest.
could not find collision_info_5 in /00000_forest.
could not find collision_info_7 in /00000_forest.
[6/4_10:28]: 297 left
extracted RGB, shape: (24, 360, 640, 3)
extracted Depth, shape: (25, 360, 640)
could not find Depth_predicted in /00000_sandbox.
extracted control_info, shape: (49, 7)
extracted collision_info, shape: (49, 

extracted RGB, shape: (758, 360, 640, 3)
extracted Depth, shape: (758, 360, 640)
could not find Depth_predicted in /00007_forest.
extracted control_info, shape: (1515, 7)
extracted collision_info, shape: (1515, 2)
could not find collision_info_3 in /00007_forest.
could not find collision_info_5 in /00007_forest.
could not find collision_info_7 in /00007_forest.
[6/4_10:30]: 276 left
extracted RGB, shape: (61, 360, 640, 3)
extracted Depth, shape: (61, 360, 640)
could not find Depth_predicted in /00007_sandbox.
extracted control_info, shape: (122, 7)
extracted collision_info, shape: (122, 2)
could not find collision_info_3 in /00007_sandbox.
could not find collision_info_5 in /00007_sandbox.
could not find collision_info_7 in /00007_sandbox.
[6/4_10:30]: 275 left
extracted RGB, shape: (730, 360, 640, 3)
extracted Depth, shape: (730, 360, 640)
could not find Depth_predicted in /00008_canyon.
extracted control_info, shape: (1460, 7)
extracted collision_info, shape: (1460, 2)
could not find

extracted Depth, shape: (32, 360, 640)
could not find Depth_predicted in /00014_sandbox.
extracted control_info, shape: (65, 7)
extracted collision_info, shape: (65, 2)
could not find collision_info_3 in /00014_sandbox.
could not find collision_info_5 in /00014_sandbox.
could not find collision_info_7 in /00014_sandbox.
[6/4_10:32]: 254 left
extracted RGB, shape: (568, 360, 640, 3)
extracted Depth, shape: (567, 360, 640)
could not find Depth_predicted in /00015_canyon.
extracted control_info, shape: (1135, 7)
extracted collision_info, shape: (1135, 2)
could not find collision_info_3 in /00015_canyon.
could not find collision_info_5 in /00015_canyon.
could not find collision_info_7 in /00015_canyon.
[6/4_10:32]: 253 left
extracted RGB, shape: (586, 360, 640, 3)
extracted Depth, shape: (585, 360, 640)
could not find Depth_predicted in /00015_forest.
extracted control_info, shape: (1170, 7)
extracted collision_info, shape: (1170, 2)
could not find collision_info_3 in /00015_forest.
could 

extracted RGB, shape: (527, 360, 640, 3)
extracted Depth, shape: (527, 360, 640)
could not find Depth_predicted in /00022_canyon.
extracted control_info, shape: (1054, 7)
extracted collision_info, shape: (1054, 2)
could not find collision_info_3 in /00022_canyon.
could not find collision_info_5 in /00022_canyon.
could not find collision_info_7 in /00022_canyon.
[6/4_10:34]: 232 left
extracted RGB, shape: (552, 360, 640, 3)
extracted Depth, shape: (553, 360, 640)
could not find Depth_predicted in /00022_forest.
extracted control_info, shape: (1105, 7)
extracted collision_info, shape: (1105, 2)
could not find collision_info_3 in /00022_forest.
could not find collision_info_5 in /00022_forest.
could not find collision_info_7 in /00022_forest.
[6/4_10:34]: 231 left
extracted RGB, shape: (78, 360, 640, 3)
extracted Depth, shape: (79, 360, 640)
could not find Depth_predicted in /00022_sandbox.
extracted control_info, shape: (157, 7)
extracted collision_info, shape: (157, 2)
could not find co

extracted RGB, shape: (751, 360, 640, 3)
extracted Depth, shape: (752, 360, 640)
could not find Depth_predicted in /00029_forest.
extracted control_info, shape: (1502, 7)
extracted collision_info, shape: (1502, 2)
could not find collision_info_3 in /00029_forest.
could not find collision_info_5 in /00029_forest.
could not find collision_info_7 in /00029_forest.
[6/4_10:36]: 210 left
extracted RGB, shape: (22, 360, 640, 3)
extracted Depth, shape: (22, 360, 640)
could not find Depth_predicted in /00029_sandbox.
extracted control_info, shape: (44, 7)
extracted collision_info, shape: (44, 2)
could not find collision_info_3 in /00029_sandbox.
could not find collision_info_5 in /00029_sandbox.
could not find collision_info_7 in /00029_sandbox.
[6/4_10:36]: 209 left
extracted RGB, shape: (967, 360, 640, 3)
extracted Depth, shape: (967, 360, 640)
could not find Depth_predicted in /00030_canyon.
extracted control_info, shape: (1934, 7)
extracted collision_info, shape: (1934, 2)
could not find c

extracted RGB, shape: (121, 360, 640, 3)
extracted Depth, shape: (121, 360, 640)
could not find Depth_predicted in /00036_sandbox.
extracted control_info, shape: (241, 7)
extracted collision_info, shape: (241, 2)
could not find collision_info_3 in /00036_sandbox.
could not find collision_info_5 in /00036_sandbox.
could not find collision_info_7 in /00036_sandbox.
[6/4_10:38]: 188 left
extracted RGB, shape: (628, 360, 640, 3)
extracted Depth, shape: (627, 360, 640)
could not find Depth_predicted in /00037_canyon.
extracted control_info, shape: (1254, 7)
extracted collision_info, shape: (1254, 2)
could not find collision_info_3 in /00037_canyon.
could not find collision_info_5 in /00037_canyon.
could not find collision_info_7 in /00037_canyon.
[6/4_10:38]: 187 left
extracted RGB, shape: (690, 360, 640, 3)
extracted Depth, shape: (691, 360, 640)
could not find Depth_predicted in /00037_forest.
extracted control_info, shape: (1380, 7)
extracted collision_info, shape: (1380, 2)
could not fi

extracted RGB, shape: (546, 360, 640, 3)
extracted Depth, shape: (546, 360, 640)
could not find Depth_predicted in /00044_canyon.
extracted control_info, shape: (1092, 7)
extracted collision_info, shape: (1092, 2)
could not find collision_info_3 in /00044_canyon.
could not find collision_info_5 in /00044_canyon.
could not find collision_info_7 in /00044_canyon.
[6/4_10:40]: 166 left
extracted RGB, shape: (632, 360, 640, 3)
extracted Depth, shape: (632, 360, 640)
could not find Depth_predicted in /00044_forest.
extracted control_info, shape: (1264, 7)
extracted collision_info, shape: (1264, 2)
could not find collision_info_3 in /00044_forest.
could not find collision_info_5 in /00044_forest.
could not find collision_info_7 in /00044_forest.
[6/4_10:40]: 165 left
extracted RGB, shape: (75, 360, 640, 3)
extracted Depth, shape: (75, 360, 640)
could not find Depth_predicted in /00044_sandbox.
extracted control_info, shape: (150, 7)
extracted collision_info, shape: (150, 2)
could not find co

In [71]:
# Request that the HDF5 library flush its buffers to disk.
f.flush()

In [75]:
# Read in hdf5 file to check if everything is there:
f = h5py.File(root_dir+'/data.hdf5', 'r')
print 'Reading data.hdf5'
for g in f.keys():
    print g
    for ds in f[g].keys():
        print '---{0}: {1}'.format(ds, f[g][ds].shape)
f.close()

Reading data.hdf5
00000_canyon
---Depth: (130, 360, 640)
---Depth_predicted: (119, 128, 160)
---RGB: (129, 360, 640, 3)
---collision_info: (129, 2)
---collision_info_3: (129, 2)
---collision_info_5: (129, 2)
---collision_info_7: (129, 2)
---control_info: (258, 7)
00001_canyon
---Depth: (96, 360, 640)
---Depth_predicted: (95, 128, 160)
---RGB: (95, 360, 640, 3)
---collision_info: (95, 2)
---collision_info_3: (95, 2)
---collision_info_5: (95, 2)
---collision_info_7: (95, 2)
---control_info: (191, 7)
